# Homework 5: Bayesian Classifiers

Follow the instructions in the template, score will be given uppon it.

### Import libraries
You are allowed to use these libraries only.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

## Part 1 - Bernoulli Naive Bayes

In [2]:
data = pd.read_csv("train_NB.csv")

In [3]:
data

,X1,X2,X3,Label
0,1,1,1,1
1,1,0,0,0
2,0,0,1,1
3,1,0,0,0
4,0,0,1,1
5,0,1,1,1
6,1,1,1,0
7,1,1,0,0
8,0,1,1,1


The data, the X and the y. 

In [4]:
X = data.drop(columns='Label').values
y = data['Label'].values


Here you should write down your code. It's ok if the code is suitable for just 2 classes.

In [16]:
class BernoulliNaiveBayes:
    
    def __init__(self):  
        pass
    
    def fit(self, X, y):
        # X and y are numpy arrays.
        # Use numpy (or pandas) the find the priors, P(y=0) and P(y=1)
        # find P(X=1|y=0) and P(X=1|y=1) for each feature.
        # The .fit method can be written in 3 lines of code. 
        prior = lambda yi: y[y==yi].shape[0] / y.shape[0]
        self.priors = np.array([prior(y_class) for y_class in sorted(np.unique(y))])
        self.X = X
        self.y = y
        
        
    def predict_proba(self, X) -> np.array:      
        # Given a new sample X, you should return a probability vector with 2 elements.
        # The probability of class 0, and the probability of class 1.
        # Given an n by p matrix, you should return an n by 2 matrix.
        # The .predict_proba method can be written in 3 lines of code. (you may also use more)
        result = []        
        X_train1 = self.X
        if type(X_train1) == pd.DataFrame:
            X_train1 = X_train1.values
        likelihood_mat = self.get_likelihood()
        
        p_xs_1 = X_train1.sum(axis=0) / X_train1.shape[0]
        
        result = []
        
        for y_class in (0,1):
            prior = self.priors[y_class]
            for i in range(X.shape[0]):
                row = []
                p_x = 1
                p_l = 1
                for j in range(X.shape[1]):
                    value = X_train1[i,j]
                    p_xj = p_xs_1[j] if value == 1 else (1 -  p_xs_1[j])
                    p_x *= p_xj

                    p_lj = likelihood_mat[y_class, j] if value == 1 else (1 - likelihood_mat[y_class, j])
                    p_l *= p_lj
                    row.append((prior*p_l) / p_x)
                result.append(row)
        return np.array(result).T
            

        

    def predict(self, X) -> np.array:
        # Given a new sample X, you should return the predicted label, which is the one with the highest probability.
        # You may use previous methods
        #TODO: your solution
        return np.max(self.predict_proba(X), axis=0)

    def get_likelihood(self) -> np.array:
        # Returns the likelihood for each x value, given the y value p(x|y).
        # Should return a matrix with shape: (#classes, #features)
        # for that matrix X[i,j] = p(X_j=1|y=i)
        X = self.X
        y = self.y
        likelihood_calc = lambda yi: (X[y==yi].sum(axis=0) / y[y==yi].shape[0])
        return np.array([likelihood_calc(y_class) for y_class in sorted(np.unique(y))])



In order to make sure you have done it right, check the probabilities for the samples [[1,0,1], [1,1,1], [0,1,1]]. <br>


In [17]:
clf = BernoulliNaiveBayes() #It is the class you just wrote down.

test = np.array([
    [1,0,1],
    [1,1,1],
    [0,1,1]]) #The test sample.

clf.fit(X,y)

print (clf.predict_proba(test))

#expected answer:
#[[ 0.55555556  0.44444444]
# [ 0.45454545  0.54545455]
# [ 0.          1.        ]]

print(clf.predict(test))

# expected answer:
# [0,1,1]

[[0.8   0.8   0.    0.2   0.2   1.   ]
 [0.72  0.9   0.    0.216 0.18  0.9  ]
 [0.27  2.025 0.    0.324 0.    1.35 ]]
[0.8   2.025 0.    0.324 0.2   1.35 ]


In [ ]:
test

array([[1, 0, 1],
       [1, 1, 1],
       [0, 1, 1]])

# Part 2 - Predicting  gender from name

### read data

In [ ]:
df = pd.read_csv("names.csv", index_col=0)
y = df.is_female
df

,name,is_female
0,Dorian,1
1,Loella,1
2,Minda,1
3,Lou,1
4,Michaeline,1
...,...,...
7939,Reza,0
7940,Samuele,0
7941,Ralina,1
7942,Aloisia,1


### generate features

In this part, you shall perform a task from the domain of Natural Language Processing (NLP).
You shall predict whether a name belongs to a male or a female. For simplicity, we shall do this only based on the last letter of the name.
In NLP many times we need to make up features using expert knowledge to be able to classify words. <br>
In the following cell you are asked to:
1. Extract the last letter of every name
2. Make a **Numpy array** `X` of dummy varialbes (one hot encoding) from the last letter feature.<br>
   (use pd.get_dummies)

In [ ]:
last_letter = pd.Series([name[-1] for name in df.name], index=df.index)

#TODO: Make binary data 
X = pd.get_dummies(last_letter)


### train and predict

Now that we have a binary data we can use the `BernoulliNaiveBayes` class you wrote to classify the names. <br>
Run the cell below to see the results, and answer the question below.
1. we plot 2 plots of the most common male/female letters ($P(X|y)$)
2. we plot the predictions ($P(y|X)$) for every letter

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6)
clf = BernoulliNaiveBayes() #It is the class you just wrote.
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
acc = (preds == y_test).mean()
print(f"accuracy: {(acc*100):.0f}%")


letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
           'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] # no name ending in `q`


#using likelihood method
likelihood_matrix = clf.get_likelihood()
p1 = likelihood_matrix[1]
p0 = likelihood_matrix[0]
# most indicating letter for each gender
female = pd.Series(p1, index=letters).sort_values(ascending=False).head(10)
male = pd.Series(p0, index=letters).sort_values(ascending=False).head(10)

# plot results
plt.bar(female.index, female*100, color='firebrick')
plt.ylabel("% of females")
plt.xlabel("last letter in name")
plt.title("top 10 last letters for females")
plt.ylim(0,40)
plt.show()

plt.bar(male.index, male*100, color='mediumblue')
plt.ylabel("% of males")
plt.ylim(0,40)
plt.xlabel("last letter in name")
plt.title("top 10 last letters for males")
plt.show()

is_letter_female = clf.predict_proba(np.eye(25))[:, 1]
is_letter_female = pd.DataFrame(is_letter_female, index = letters, columns=['female_proba'])
is_letter_female = is_letter_female.sort_values('female_proba', ascending=False)
is_letter_female.style.background_gradient(cmap='coolwarm')

C:\Users\itayz\AppData\Local\Temp\ipykernel_12420\2169484599.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  acc = (preds == y_test).mean()


AttributeError: 'bool' object has no attribute 'mean'

### question
explain how come the letter *i* has higher probability to be feminine than *y* even though it is less popular for women?

**Answer:** <br>


In [ ]:
all_male = pd.Series(p0, index=letters).sort_values(ascending=False)
print(female.y)
print(all_male.i)


0.09281914893617021
0.0186533212010919


it is probebly because the letter `i` is in the top ten of `female`, but not in the top ten of `male`. unlike `y` which is popular in both groups.

in fact `i` is almost ten times more popular with `female` compared to `male`.  
this makes `i` stronger in distinguishing between `male` and `female`.